In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.preprocessing import MinMaxScaler

# --- 1. SETUP & MOCK DATA ---
# This simulates the data environment found in your notebooks
def setup_benchmark_data():
    # Core nutrients used across all models
    nutrients = ['Energ_Kcal', 'Protein_(g)', 'Fiber_TD_(g)', 'Iron_(mg)', 'Potassium_(mg)']
    
    # Mocking vegetable dataset
    veg_data = pd.DataFrame({
        'NDB_No': range(100, 105),
        'Shrt_Desc': ['CABBAGE, RAW', 'ACEROLA JUICE, RAW', 'CASSAVA, RAW', 'BEANS, SNAP, RAW', 'MORINGA LEAVES, RAW'],
        'Energ_Kcal': [25, 32, 160, 31, 64],
        'Protein_(g)': [1.3, 0.4, 1.4, 1.8, 9.4],
        'Fiber_TD_(g)': [2.5, 0.3, 1.8, 3.4, 2.0],
        'Iron_(mg)': [0.5, 0.2, 0.3, 1.0, 4.0],
        'Potassium_(mg)': [170, 146, 271, 209, 337],
        'is_priority': [False, False, True, True, True] # Simulating CotD Priority
    })
    
    # User RDA Target (e.g., for an Adult Male in Hambantota)
    target_rda = np.array([500, 15, 10, 3, 500]) # Scaled targets for a single meal
    
    return veg_data, target_rda, nutrients

# --- 2. MODEL WRAPPERS ---

def run_heuristic_model(data, target, nutrients):
    """Simulates code.ipynb: Weighted Cosine + Rules"""
    # Heuristic: Iron is weighted heavily for Anemia
    weights = np.array([1.0, 1.0, 1.0, 4.0, 1.0]) 
    scores = cosine_similarity(target.reshape(1, -1) * weights, data[nutrients] * weights)[0]
    # Apply local priority boost
    scores += data['is_priority'].astype(float) * 0.2
    return scores

def run_spn_model(data, target, nutrients):
    """Simulates code_spn.ipynb: Probabilistic Likelihood"""
    # SPN finds the likelihood of a sample belonging to the distribution
    # Here we simulate higher probability for nutrient-dense outliers
    z_scores = np.abs((data[nutrients] - data[nutrients].mean()) / data[nutrients].std()).mean(axis=1)
    return 1 / (1 + z_scores)

def run_xgboost_model(data, target, nutrients):
    """Simulates code_xgboost.ipynb: Learned Ranking Patterns"""
    # XGBoost learns that Sri Lankan users favor Energy and Fiber
    return (data['Fiber_TD_(g)'] * 0.5) + (data['Energ_Kcal'] * 0.3)

# --- 3. EVALUATION SUITE ---

def compare_models():
    veg_df, rda_target, nut_cols = setup_benchmark_data()
    
    models = {
        "Heuristic (Rules)": run_heuristic_model,
        "SPN (Probabilistic)": run_spn_model,
        "XGBoost (LTR)": run_xgboost_model
    }
    
    results = []
    
    print("="*60)
    print("AI MODEL COMPARISON: SRI LANKAN VEGETABLE RECOMMENDATION")
    print("="*60)

    for name, model_func in models.items():
        print(f"\nEvaluating {name}...")
        
        # Get scores and top 3 recommendations
        scores = model_func(veg_df, rda_target, nut_cols)
        top_indices = np.argsort(scores)[-3:][::-1]
        top_recs = veg_df.iloc[top_indices]
        
        # Metric 1: Nutritional Error (RMSE)
        # $RMSE = \sqrt{\frac{1}{n} \sum (Target - Actual)^2}$
        errors = []
        for _, row in top_recs.iterrows():
            err = np.sqrt(np.mean((rda_target - row[nut_cols].values)**2))
            errors.append(err)
        avg_rmse = np.mean(errors)
        
        # Metric 2: Estimated Cost (Simulated LKR)
        avg_cost = top_recs['Energ_Kcal'].mean() * 5 # Simplified: Cost proportional to calories
        
        # Metric 3: Intra-List Diversity
        # (1 - average cosine similarity between top 3 items)
        if len(top_recs) > 1:
            dist_matrix = cosine_distances(top_recs[nut_cols])
            diversity = np.mean(dist_matrix[np.triu_indices(len(top_recs), k=1)])
        else:
            diversity = 0
            
        # Metric 4: Priority Match Rate
        priority_rate = top_recs['is_priority'].mean() * 100

        results.append({
            "Model": name,
            "Nutrient Error (RMSE)": round(avg_rmse, 2),
            "Avg Cost (LKR)": round(avg_cost, 2),
            "Diversity Score": round(diversity, 4),
            "Priority Match (%)": priority_rate
        })
        
        print(f"  - Top Pick: {top_recs.iloc[0]['Shrt_Desc']}")
        print(f"  - RMSE: {avg_rmse:.2f} (Lower is better)")
        print(f"  - Diversity: {diversity:.4f} (Higher is better)")

    # Display Summary Table
    print("\n" + "="*60)
    print("FINAL COMPARISON SUMMARY")
    print("="*60)
    summary_df = pd.DataFrame(results)
    print(summary_df.to_string(index=False))

# Run the suite
compare_models()

AI MODEL COMPARISON: SRI LANKAN VEGETABLE RECOMMENDATION

Evaluating Heuristic (Rules)...
  - Top Pick: CASSAVA, RAW
  - RMSE: 212.87 (Lower is better)
  - Diversity: 0.0447 (Higher is better)

Evaluating SPN (Probabilistic)...
  - Top Pick: BEANS, SNAP, RAW
  - RMSE: 229.72 (Lower is better)
  - Diversity: 0.0493 (Higher is better)

Evaluating XGBoost (LTR)...
  - Top Pick: CASSAVA, RAW
  - RMSE: 212.87 (Lower is better)
  - Diversity: 0.0447 (Higher is better)

FINAL COMPARISON SUMMARY
              Model  Nutrient Error (RMSE)  Avg Cost (LKR)  Diversity Score  Priority Match (%)
  Heuristic (Rules)                 212.87           425.0           0.0447          100.000000
SPN (Probabilistic)                 229.72           360.0           0.0493           66.666667
      XGBoost (LTR)                 212.87           425.0           0.0447          100.000000
